In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import svm

def compute_GridSearchCV(kernels, regularizations, gamma='auto', cv=4, random_state=0):
    # Load the Iris dataset
    iris = load_iris()
    X, y = iris.data, iris.target

    # Split the dataset into 70% train and 30% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Define the parameter grid
    param_grid = {
        'kernel': kernels,
        'C': regularizations,
        'gamma': [gamma]
    }

    # Initialize the model (SVC) with given random state
    model = svm.SVC(random_state=random_state)

    # Initialize GridSearchCV with cross-validation = 4
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv)
    grid.fit(X_train, y_train)

    # Return the mean cross-validated score of the best estimator
    return grid.best_score_

# Hyperparameters as specified
kernels = ['linear', 'rbf']
regularizations = [1, 15, 25]

# Compute the mean cross-validated score using the function
best_score = compute_GridSearchCV(kernels, regularizations, gamma='auto', cv=4, random_state=0)
print("Best cross-validated score: ", best_score)


Best cross-validated score:  0.9807692307692308


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Read the dataset.
# The Social_Network_Ads dataset is assumed to have a header row.
df = pd.read_csv("/content/drive/MyDrive/MLP/GA/Week8/Social_Network_Ads.csv")
print("Dataset head:")
print(df.head())

# For this assignment we use "Age" and "EstimatedSalary" as features
# and "Purchased" as the target.
X = df[['Age', 'EstimatedSalary']]
y = df['Purchased']

# Split the dataset into training and test sets (75:25 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Fit and transform the training features and transform the test features using StandardScaler.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit a linear SVM (SVC with linear kernel and random_state=0) on the training data.
model = SVC(kernel='linear', random_state=0)
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Calculate the accuracy score
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy score on test set:", acc)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

Dataset head:
   Age  EstimatedSalary  Purchased
0   19            19000          0
1   35            20000          0
2   26            43000          0
3   27            57000          0
4   19            76000          0

Accuracy score on test set: 0.9

Confusion Matrix:
[[66  2]
 [ 8 24]]


In [ ]:
# Part 1: Linear SVC with MinMaxScaler on MNIST (first 20,000 for training, next 5,000 for testing)
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

# Fetch MNIST (this may take a moment)
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]

# Convert targets to integers
y = y.astype(np.int8)

# Select first 20,000 samples as training and next 5,000 as test
X_train_part = X[:20000]
y_train_part = y[:20000]
X_test_part = X[20000:25000]
y_test_part = y[20000:25000]

# Build a pipeline with MinMaxScaler and SVC with linear kernel, decision_function_shape='ovr'
pipe_linear = make_pipeline(MinMaxScaler(),
                            SVC(kernel='linear', decision_function_shape='ovr', class_weight=None, random_state=0))

# Fit the pipeline on the training data and predict on test data
pipe_linear.fit(X_train_part, y_train_part)
y_pred_linear = pipe_linear.predict(X_test_part)

# Compute the confusion matrix and sum of diagonal elements
cm = confusion_matrix(y_test_part, y_pred_linear)
diag_sum = np.trace(cm)
print("Confusion Matrix for Linear SVC:")
print(cm)
print("Sum of main diagonal elements:", diag_sum)

# Compute overall precision, recall, and f1-score (macro averaging gives an overall idea,
# but here we show the weighted metrics as well for consistency)
prec = precision_score(y_test_part, y_pred_linear, average='weighted')
rec = recall_score(y_test_part, y_pred_linear, average='weighted')
f1 = f1_score(y_test_part, y_pred_linear, average='weighted')
print("\nEvaluation metrics for Linear SVC on MNIST:")
print(f"Precision: {prec:.2f}, Recall: {rec:.2f}, F1 Score: {f1:.2f}")

# Expected answers (closest options):
# Q4 (sum of diagonal): 4623
# Q5 (Precision/Recall/F1): 0.92, 0.92, 0.92

###############################################################################
# Part 2: Poly-kernel SVM with StandardScaler on MNIST (50:50 train-test split)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the MNIST dataset into 50:50 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Build a pipeline with StandardScaler and SVC with poly kernel and specified parameters
pipe_poly = make_pipeline(StandardScaler(),
                          SVC(kernel='poly', degree=3, decision_function_shape='ovr',
                              class_weight='balanced', C=10, random_state=0))

# Fit the pipeline and predict on test data
pipe_poly.fit(X_train, y_train)
y_pred_poly = pipe_poly.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred_poly, output_dict=True)
weighted_f1 = report['weighted avg']['f1-score']
print("\nClassification Report for Poly-kernel SVC:")
print(classification_report(y_test, y_pred_poly))
print("Weighted Average F1 Score:", weighted_f1)

# Expected answer for weighted avg f1-score: around 0.97

Confusion Matrix for Linear SVC:
[[469   0   1   0   0   5   1   1   1   0]
 [  1 555   2   4   0   1   1   1   3   0]
 [  5   4 479   6   9   1   3   6   7   1]
 [  3   3  13 462   0  19   0   3   9   4]
 [  2   0   6   0 468   1   2   4   1  16]
 [ 14   0   0  18   2 405   3   2   7   9]
 [  1   2   2   0   6   9 471   0   0   0]
 [  0   2   4   1   7   1   0 483   0   6]
 [  3  10  11  15   0  19   6   3 396   3]
 [  3   6   1   2  21   0   0  26   2 435]]
Sum of main diagonal elements: 4623

Evaluation metrics for Linear SVC on MNIST:
Precision: 0.92, Recall: 0.92, F1 Score: 0.92


In [1]:
# Import necessary modules
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Function 1: Using poly kernel, C=10, gamma='auto'
def compute_score_poly(X_train, y_train, X_test, y_test):
    # Create the model with poly kernel, regularization parameter 10, gamma 'auto'
    model = SVC(kernel='poly', C=10, gamma='auto', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Function 2: Using sigmoid kernel, C=25, gamma='auto'
def compute_score_sigmoid(X_train, y_train, X_test, y_test):
    # Create the model with sigmoid kernel, regularization parameter 25, gamma 'auto'
    model = SVC(kernel='sigmoid', C=25, gamma='auto', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Function 3: Build a pipeline (Scaler and SVC), dropping Iris-setosa rows and computing precision
def compute_precision_score():
    # Load the Iris dataset
    iris = load_iris()
    # Create a DataFrame for easier manipulation
    import pandas as pd
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    # In Iris: 0 = Iris-setosa, 1 = Iris-versicolor, 2 = Iris-virginica.
    # Drop rows where target is 0 (Iris-setosa)
    df_filtered = df[df['target'] != 0]

    # Features and target for the filtered dataset
    X = df_filtered[iris.feature_names]
    y = df_filtered['target']

    # Create a pipeline with MinMaxScaler (named Scaler) and SVC (default parameters)
    pipeline = make_pipeline(MinMaxScaler(), SVC(random_state=0))

    # Split data into 75:25 ratio with random_state=0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

    # Fit the pipeline and predict
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Compute the weighted precision score
    return precision_score(y_test, y_pred, average='weighted')

# Main block: Split the Iris dataset for the first two functions
iris = load_iris()
X = iris.data
y = iris.target

# Split data: 70% training, 30% test, random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Compute and print scores for each part
accuracy_poly = compute_score_poly(X_train, y_train, X_test, y_test)
accuracy_sigmoid = compute_score_sigmoid(X_train, y_train, X_test, y_test)
precision_val = compute_precision_score()

print("Accuracy score using poly kernel (C=10, gamma='auto'):", accuracy_poly)
print("Accuracy score using sigmoid kernel (C=25, gamma='auto'):", accuracy_sigmoid)
print("Weighted Precision score (after dropping Iris-setosa):", precision_val)


Accuracy score using poly kernel (C=10, gamma='auto'): 1.0
Accuracy score using sigmoid kernel (C=25, gamma='auto'): 0.28888888888888886
Weighted Precision score (after dropping Iris-setosa): 0.9314285714285714
